In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import astropy.io.fits as pyfits
import astropy.utils as autils
import requests
import json
import datetime
from pprint import pprint as pp
import os
import astropy.io.fits as fits
import astropy.io.ascii as ascii
import copy

# Jielai added modules f
import subprocess
import seaborn as sns

# Time Counter function
import time
def tic():
    tic.start = time.perf_counter()
def toc():
    elapsed_seconds = time.perf_counter() - tic.start
    return elapsed_seconds # fractional

# NOAO server Settings
natroot = 'https://astroarchive.noirlab.edu'
assert natroot == 'https://astroarchive.noirlab.edu', 'Notebook does NOT point to PRODUCTION'
print(f"Using server on {natroot}")
adsurl = f'{natroot}/api/adv_search'
print(f"adsurl = {adsurl}")
apiurl = f'{adsurl}/fasearch/?limit=200000'
print(f'Using API url: {apiurl}')

# Start the timer
print(f'Started on: {str(datetime.datetime.now())}')
tic() # Start timing the run of this notebook

Using server on https://astroarchive.noirlab.edu
adsurl = https://astroarchive.noirlab.edu/api/adv_search
Using API url: https://astroarchive.noirlab.edu/api/adv_search/fasearch/?limit=200000
Started on: 2022-02-18 14:27:16.195440


In [2]:
def  get_radec_maxmin(RAcentre,DECcentre,search_radius_deg, debug=False):

    dec_min = DECcentre - search_radius_deg
    dec_max = DECcentre + search_radius_deg
    if dec_min<-90.0: dec_min=-90.0
    if dec_max>90.0: dec_max=90.0
    if dec_min==-90.0 or dec_max==90.0:
        ra_min = 0
        ra_max = 360.0
    else:
        costerm = min(np.cos(dec_min*np.pi/180.0),np.cos(dec_max*np.pi/180.0))
        ra_min = RAcentre-search_radius_deg*1./costerm
        ra_max = RAcentre+search_radius_deg*1./costerm
        if ra_min<0: ra_min+=360.0
        if ra_max>360.0: ra_max-=360.0

    if debug:
        print('**** DEBUG: ',dec_min, dec_max)
        print('**** DEBUG: ',ra_min, ra_max)

    return ra_min,ra_max,dec_min,dec_max
def makedirs_(out_path):
    out_dir = '/'.join(out_path.split('/')[0:-1])
    os.makedirs(out_dir, exist_ok=True)
    return None

In [3]:
pw = 'XXX'
usrname = 'XXX'
search_radius_deg = 0.5
caldat1g = '2022-02-01'
caldat2g = '2022-02-23'
raw_dir = '/Users/jielaizhang/Desktop/' # where downloaded stuff is saved
RA  = 203.6800149
DEC = 33.2167887

In [5]:
ra_min,ra_max,dec_min,dec_max = get_radec_maxmin(RA,DEC,search_radius_deg,debug=False)
if ra_min>ra_max:
    raise RuntimeError('This needs to be fixed!!')
    # Need to perform two searches to account for overlap

# Json needed to make a request to NOAO archive. 
jj = {    
        "outfields" : [
            "md5sum","release_date","proposal","archive_filename","original_filename","url",
            "proc_type","prod_type","ifilter","exposure",
            "ra_center","dec_center","dateobs_min","dateobs_max","caldat","filesize",
            "seeing"
            ],
        "search" : [ ["instrument", 'decam'],["telescope", 'ct4m'],["obs_type", 'object'],
                     ['ra_center',ra_min,ra_max],['dec_center',dec_min,dec_max],
                     ["caldat","1000-01-01",caldat2g],
                     ["proc_type","instcal","stacked"],['prod_type','image']
                   ]
    }
ads_df = pd.DataFrame(requests.post(apiurl,json=jj).json()[1:])
print('Retrieved: ',len(ads_df))
if len(ads_df) != 0:
    print(np.unique(ads_df['caldat']))
    print(np.unique(ads_df['exposure']))

Retrieved:  22
['2022-02-14' '2022-02-15' '2022-02-16']
[ 30.  60. 120.]


In [14]:
pd.set_option('display.max_colwidth', -1)
ads_df[['archive_filename','proc_type','exposure','caldat','seeing','url']]

,archive_filename,proc_type,exposure,caldat,seeing,url
0,/net/archive/pipe/20220216/ct4m/2022A-679480/c4d_220217_085447_ooi_i_kntrap.fits.fz,instcal,30.0,2022-02-16,1.57,https://astroarchive.noirlab.edu/api/retrieve/004ac61b728a69a27bf0e34c1bbbaa11/
1,/net/archive/pipe/20220215/ct4m/2022A-679480/c4d_220216_090019_ooi_r_kntrap.fits.fz,instcal,30.0,2022-02-15,1.44,https://astroarchive.noirlab.edu/api/retrieve/15a0925821821066ce09f5125af739e0/
2,/net/archive/pipe/20220216/ct4m/2022A-679480/c4d_220217_085716_ooi_z_kntrap.fits.fz,instcal,60.0,2022-02-16,1.30,https://astroarchive.noirlab.edu/api/retrieve/17d3653b08c7f07b42cfd4fd94cd75a6/
3,/net/archive/pipe/20220216/ct4m/2022A-679480/c4d_220217_085546_ooi_z_kntrap.fits.fz,instcal,60.0,2022-02-16,1.30,https://astroarchive.noirlab.edu/api/retrieve/18fbba5961892f2b41c4a807d79195a9/
4,/net/archive/pipe/20220214/ct4m/2022A-679480/c4d_220215_084336_ooi_g_v1.fits.fz,instcal,30.0,2022-02-14,1.67,https://astroarchive.noirlab.edu/api/retrieve/215594b186cf1aae8dfd865bbe3c32e9/
5,/net/archive/pipe/20220214/ct4m/2022A-679480/c4d_220215_084336_ooi_g_kntrap.fits.fz,instcal,30.0,2022-02-14,1.67,https://astroarchive.noirlab.edu/api/retrieve/2f1a05d33f7c13234640ad4589053abd/
6,/net/archive/pipe/20220216/ct4m/2022A-679480/c4d_220217_085149_ooi_g_kntrap.fits.fz,instcal,30.0,2022-02-16,1.80,https://astroarchive.noirlab.edu/api/retrieve/3fed02f69615ce229e54cd162f770e87/
7,/net/archive/pipe/20220215/ct4m/2022A-679480/c4d_220216_090019_ooi_r_v1.fits.fz,instcal,30.0,2022-02-15,1.44,https://astroarchive.noirlab.edu/api/retrieve/447adf09c5b365f137c26d40b49517dc/
8,/net/archive/pipe/20220216/ct4m/2022A-679480/c4d_220217_085546_osi_z_kntrap.fits.fz,stacked,120.0,2022-02-16,NaN,https://astroarchive.noirlab.edu/api/retrieve/46bc0872f069bc8c012a030e72495058/
9,/net/archive/pipe/20220216/ct4m/2022A-679480/c4d_220217_085149_osi_g_kntrap.fits.fz,stacked,60.0,2022-02-16,NaN,https://astroarchive.noirlab.edu/api/retrieve/527083c1726f0db0c22b283a8ba5421c/
